### Let's load data to see what we are dealing with ###

In [1]:
import numpy as np
import pandas as pd
#import sys  
#reload(sys) 
#sys.setdefaultencoding('utf8')

In [2]:
train_data=pd.read_csv('train.csv',encoding='utf-8')
test_data=pd.read_csv('test.csv',encoding='utf-8')

In [3]:
train_data.iloc[629]

id                                                  id16017
text      "C'est à vous à faire," said his Majesty, cutt...
author                                                  EAP
Name: 629, dtype: object

In [4]:
test_data.head(3)

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...


### Lets see the size, and if we have missing values in the datasets ###

In [5]:
print(train_data.shape,test_data.shape)
print(train_data.isnull().sum())
print(test_data.isnull().sum())

((19579, 3), (8392, 2))
id        0
text      0
author    0
dtype: int64
id      0
text    0
dtype: int64


### Ok, seems clean ###


### Lets make categories of authors ###

In [6]:
authors=train_data.author.unique()
dic={}
for i,author in enumerate(authors):
    dic[author]=i
labels=train_data.author.apply(lambda x:dic[x])

### Before tokenizing lets get rid of punctuations and stop words ###

In [7]:
import nltk
from nltk.stem import porter
from nltk.tokenize import RegexpTokenizer

from nltk.corpus import stopwords

In [8]:
lemma = nltk.wordnet.WordNetLemmatizer()

tokenizer = RegexpTokenizer(r'\w+')
stops=set(stopwords.words('english'))
def sentence_preproc(sentence):
    return " ".join([lemma.lemmatize(word) for word in tokenizer.tokenize(sentence) 
                     if word not in stops]).encode('ascii', errors='ignore')

In [9]:
train_data['processed_text']=train_data['text'].apply(lambda x:sentence_preproc(x))
test_data['processed_text']=test_data['text'].apply(lambda x:sentence_preproc(x))

### Lets divide our training data to train and validation ###

In [10]:
val_data=train_data.sample(frac=0.2,random_state=200)
train_data=train_data.drop(val_data.index)

### Tokenize text of the training data with keras text preprocessing functions ###

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [12]:
texts=train_data.processed_text

In [13]:
NUM_WORDS=20000
tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',
                      lower=True)
tokenizer.fit_on_texts(texts)
sequences_train = tokenizer.texts_to_sequences(texts)
sequences_valid=tokenizer.texts_to_sequences(val_data.processed_text)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 20682 unique tokens.


In [14]:
X_train = pad_sequences(sequences_train)
X_val = pad_sequences(sequences_valid,maxlen=X_train.shape[1])
y_train = to_categorical(np.asarray(labels[train_data.index]))
y_val = to_categorical(np.asarray(labels[val_data.index]))
print('Shape of X train and X validation tensor:', X_train.shape,X_val.shape)
print('Shape of label train and validation tensor:', y_train.shape,y_val.shape)



('Shape of X train and X validation tensor:', (15663, 461), (3916, 461))
('Shape of label train and validation tensor:', (15663, 3), (3916, 3))


### word embedding ###

### lets load the pretrain Word2Vec model from Google https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit ###
### It might take time since it contains contains 300-dimensional vectors for 3 million words and phrases ###

In [15]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

EMBEDDING_DIM=300
vocabulary_size=min(len(word_index)+1,NUM_WORDS)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

### Without pretrained data we can just initalize embedding matrixs as: ###

In [ ]:
from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM)

In [100]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
sequence_length = X_train.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.7



inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[2], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

maxpool_0 = MaxPooling2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPooling2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1), strides=(1,1))(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((3*num_filters,))(flatten)
dropout = Dropout(drop)(flatten)
output = Dense(units=3, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)
del (model)
# this creates a model that includes
model = Model(inputs, output)

In [39]:
from keras import backend as K

def multiclass_log_loss(y_true, y_pred, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    https://www.kaggle.com/wiki/MultiClassLogLoss

    idea from this post:
    http://www.kaggle.com/c/emc-data-science/forums/t/2149/is-anyone-noticing-difference-betwen-validation-and-leaderboard-error/12209#post12209

    Parameters
    ----------
    y_true : array, shape = [n_samples]
    y_pred : array, shape = [n_samples, n_classes]

    Returns
    -------
    loss : float
    """
    predictions = np.clip(y_pred, eps, 1 - eps)

    # normalize row sums to 1
    #predictions /= predictions.sum(axis=1)[:, np.newaxis]

    vsota = -np.sum(y_true * np.log(predictions))
    return -1.0 / y_pred.shape[0] * vsota

In [71]:
dir(K)

['Function',
 '_BACKEND',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '__path__',
 '_backend',
 '_config',
 '_config_path',
 '_epsilon',
 '_floatx',
 '_image_data_format',
 '_keras_base_dir',
 '_keras_dir',
 'abs',
 'absolute_import',
 'all',
 'any',
 'arange',
 'argmax',
 'argmin',
 'backend',
 'batch_dot',
 'batch_flatten',
 'batch_get_value',
 'batch_normalization',
 'batch_set_value',
 'bias_add',
 'binary_crossentropy',
 'cast',
 'cast_to_floatx',
 'categorical_crossentropy',
 'clear_session',
 'clip',
 'common',
 'concatenate',
 'constant',
 'control_flow_ops',
 'conv1d',
 'conv2d',
 'conv2d_transpose',
 'conv3d',
 'cos',
 'count_params',
 'ctc',
 'ctc_batch_cost',
 'ctc_decode',
 'ctc_label_dense_to_sparse',
 'cumprod',
 'cumsum',
 'defaultdict',
 'dot',
 'dropout',
 'dtype',
 'elu',
 'epsilon',
 'equal',
 'eval',
 'exp',
 'expand_dims',
 'eye',
 'flatten',
 'floatx',
 'foldl',
 'foldr',
 'function',
 'functional_ops',
 'gather',
 'get_session',
 'get

In [101]:
from keras import backend as K
def custom_loss(y_true,y_pred):
    eps=1e-15
    predictions = K.clip(y_pred, eps, 1. - eps)
    sum1=K.sum(y_true*K.log(predictions),axis=1)
    return -1.0*K.mean(sum1)
    

In [41]:
test=K.variable(value=y_val)

In [54]:
dir(K)

['Function',
 '_BACKEND',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '__path__',
 '_backend',
 '_config',
 '_config_path',
 '_epsilon',
 '_floatx',
 '_image_data_format',
 '_keras_base_dir',
 '_keras_dir',
 'abs',
 'absolute_import',
 'all',
 'any',
 'arange',
 'argmax',
 'argmin',
 'backend',
 'batch_dot',
 'batch_flatten',
 'batch_get_value',
 'batch_normalization',
 'batch_set_value',
 'bias_add',
 'binary_crossentropy',
 'cast',
 'cast_to_floatx',
 'categorical_crossentropy',
 'clear_session',
 'clip',
 'common',
 'concatenate',
 'constant',
 'control_flow_ops',
 'conv1d',
 'conv2d',
 'conv2d_transpose',
 'conv3d',
 'cos',
 'count_params',
 'ctc',
 'ctc_batch_cost',
 'ctc_decode',
 'ctc_label_dense_to_sparse',
 'cumprod',
 'cumsum',
 'defaultdict',
 'dot',
 'dropout',
 'dtype',
 'elu',
 'epsilon',
 'equal',
 'eval',
 'exp',
 'expand_dims',
 'eye',
 'flatten',
 'floatx',
 'foldl',
 'foldr',
 'function',
 'functional_ops',
 'gather',
 'get_session',
 'get

In [59]:
float(K.int_shape(test)[0])

3916.0

In [102]:
adam = Adam(lr=1e-3)

model.compile(loss=custom_loss,#'categorical_crossentropy',
              optimizer=adam,
              metrics=['acc'])
callbacks = [EarlyStopping(monitor='val_loss')]
model.fit(X_train, y_train, batch_size=100, epochs=10, verbose=1, validation_data=(X_val, y_val),
         callbacks=callbacks)  # starts training




Train on 15663 samples, validate on 3916 samples
Epoch 1/10
15663/15663 [==============================] - 342s - loss: 0.2538 - acc: 0.9623 - val_loss: 0.6597 - val_acc: 0.8105

### now lets use our model to predict test data ###

In [103]:
sequences_test=tokenizer.texts_to_sequences(test_data.processed_text)
X_test = pad_sequences(sequences_test,maxlen=X_train.shape[1])
y_pred=model.predict(X_test)

In [104]:
to_submit=pd.DataFrame(index=test_data.id,data={'EAP':y_pred[:,dic['EAP']],
                                                'HPL':y_pred[:,dic['HPL']],
                                                'MWS':y_pred[:,dic['MWS']]})

In [105]:
to_submit.to_csv('submit.csv')

In [106]:
y_val_pred=model.predict(X_val)

In [107]:
y_val.shape

(3916, 3)

In [108]:
multiclass_log_loss(y_val,y_val_pred)

-0.56256108819665329